In [ ]:
from stravalib.client import Client
from stravalib.model import Activity
import csv
import datetime
import os
import sys
import inspect
import datetime
currentdir = os.path.dirname(os.path.abspath(
    inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Include paths for module search
sys.path.insert(0, parentdir)
from db import ChallengeDB
from model import Run, Runner

# Required ENV
CLIENT_ID = os.environ["STRAVA_CLIENT_ID"]
CLIENT_SECRET = os.environ["STRAVA_CLIENT_SECRET"]

MONGODB_URI = os.environ["MONGODB_URI"]
DATABASE_NAME = os.environ["DATABASE_NAME"]

In [ ]:
now = datetime.datetime.now()
timestamp = now.strftime("%Y-%b-%d %H:%M:%S")
report_postfix = now.strftime("%Y%%d_%H%M%S")
print(timestamp)

In [ ]:
ChallengeDB.init(MONGODB_URI, DATABASE_NAME)

In [ ]:
runs = ChallengeDB.find_run()

In [ ]:
runners = ChallengeDB.find_runner()

In [ ]:
run_report_path = "run_report_%s.csv" % (report_postfix)
with open(run_report_path, 'w', newline='') as csvfile:
    fieldnames = ["timestamp"] + list(runs[0].to_doc().keys())
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for run in runs:
        row = run.to_doc()
        row["timestamp"] = timestamp
        writer.writerow(row)

In [ ]:
runner_report_path = "runner_report_%s.csv" % (report_postfix)
with open(runner_report_path, 'w', newline='') as csvfile:
    fieldnames = ["timestamp","_id", "displayname", "intania", "createdAt"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for runner in runners:
        row = runner.to_doc()
        row["timestamp"] = timestamp
        row = {key:row[key] for key in fieldnames}
        writer.writerow(row)

## Update Run

In [ ]:
## Update Run
CHALLENGE_START_DATE = datetime.datetime(2018, 10, 18, 17, 0)
CHALLENGE_END_DATE = datetime.datetime(2019, 1, 11, 17, 0)

In [ ]:
ChallengeDB.init(MONGODB_URI, DATABASE_NAME)
# Read all runners that have intania from DB
print("Get all runners from db")
runners = ChallengeDB.find_runner()
n_runner = len(runners)
print("Total runners: %d" % (n_runner))
# For each runners get their activities
runs = []
for idx, runner in enumerate(runners):
    if runner.intania is None:
        print("Skip runner with None intania: id=%s displayname='%s'" % (runner.id, runner.displayname))
        continue
    client = Client(access_token=runner.access_token)
    activities = client.get_activities(after=CHALLENGE_START_DATE, before=CHALLENGE_END_DATE)
    print("Get activities: idx=%d/%d id=%s displayname='%s' intania:'%s'" % (idx, n_runner-1, runner.id, runner.displayname, runner.intania))
    for act in activities:
        if act.type != Activity.RUN:
            continue
        run = Run.from_activity(act, runner.intania)
        # Try to save activity to DB
        if ChallengeDB.find_one_run({"_id":run.id}) is None:
            # New run activity
            ChallengeDB.insert_run(run)
        runs.append(run)
print("Total run activities: %d" % (len(runs)))